<a href="https://colab.research.google.com/github/joostnjns/Google-Earth-Engine/blob/master/SVM_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

// Query satellite image  26.996813, 14.395989
var aoi = ee.Geometry.Rectangle([14.38, 26.98, 14.49, 27.07]);
var collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);

//var rgbImage = collection.sort('CLOUD_COVER').first().select(['B3', 'B2', 'B1']);
var image_ = collection.sort('CLOUD_COVER').first();

// Define the visualization parameters.
var vizParams = {
  bands: ['B3', 'B2', 'B1'],
  min: 0,
  max: 5000,
  gamma: [0.95, 1.1, 1]
};

// Visualize satellite image              
Map.setCenter(14.42, 27.04, 13);
Map.addLayer(image_.clip(aoi), vizParams, 'Landsat 5');

// Load training data
var trainData = ee.FeatureCollection('users/jneujens/trainData');
//Map.addLayer(trainData);

// Property containing land cover labels
var label = 'classname';

// Get training from polygons on image
var training = image_.sampleRegions({
  collection: trainData,
  properties: ['classvalue'],
  scale: 30
});

// Create an SVM classifier with custom parameters.
var classifier = ee.Classifier.svm({
  decisionProcedure: 'Voting',
  svmType: 'C_SVC',
  kernelType: 'LINEAR',
  cost: 0.1
});

// Train the classifier.
var trained = classifier.train(training, 'classvalue', ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);

// Classify the image.
var classified = image_.clip(aoi).classify(trained);

// Display result
Map.addLayer(classified,
             {min: 0, max: 2, palette: ['grey', 'green','red']},
             'LC classification');
